# Actity Recognition From RGB-D Images

In [6]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import lightnet as ln
import os
import xml.etree.cElementTree as ET
from fastai.vision import *
from fastai.metrics import error_rate

## Load Data Functions

In [70]:
dataset_path = "/storage/UTKinect_Dataset"
folders = ["depth","rgb"]
label_fname = "actionLabel.txt"

In [147]:
def string_Arr_to_float(str_arr):
    """
    func: convert all string numbers in list to float
    param:  list of string numbers
    out:  list of float numbers
    """
    for x in range(len(str_arr)):
        try:
            str_arr[x] = float(str_arr[x])
    return str_arr

In [178]:
def depth_xml_to_arr(depthFilePath,beginFrm, endFrm, label):
    """
    func: load depth image data from given range of frames, and append label
    param: depthFilePath - path to file where frames are found
           beginFrm - first frame of activty 
           endFrm - last frame of activty
           label - label of activity
    out: list of touples contatining depth data of each frame between beginFrm and endFrm and label
    """
    xmlFiles = [f for f in os.listdir(depthFilePath) if f.endswith('.xml')]
    xmlFiles.sort(key=lambda item: (len(item), item))
    depthSequence = []
    
    beginFrmFile = "depthImg%i.xml" %beginFrm
    endFrmFile = "depthImg%i.xml" %endFrm
    
    beginIdx = xmlFiles.index(beginFrmFile)
    endIdx = xmlFiles.index(endFrmFile)
    useXmlFiles = xmlFiles[beginIdx:endIdx+1]
    
    for xmlFile in useXmlFiles:
        try:
            tree = ET.parse(os.path.join(depthFilePath, xmlFile))
            filename, _ = os.path.splitext(xmlFile)
            elem = tree.find('%s/data' % filename)
            strData = elem.text
            str_Arr = strData.split()
            floatData = string_Arr_to_float(str_Arr)
            arr = np.array(floatData)
            depthData = np.array(floatData).reshape(240, 320)
            depthSequence.append((depthData,label))
        except:
            print("Problem in: %s" %xmlFile)
    return depthSequence

In [179]:
def load_depth_imgs(depth_path,label_fname):
    """
    func: load depth image data from xml file
    param: depth_path - path to folder with depth data
           label_fname - name of the label txt file
    out: list of touples with depth img and activity label
    """
    depth_data = []
    
    configFile = os.path.join(dataset_path, label_fname)
    with open(configFile, "r") as f:
        lines = f.read().splitlines()

    for i in range(0,220,11):
        currLines = lines[i:i+11]
        depthFileDir = currLines[0]

        print("loading %s" %depthFileDir)
        for line in currLines[1:]:
                splitIdx = line.index(":")
                label = line[:splitIdx]
                nums = line[splitIdx+1:].split()
                beginFrm = int(nums[0])
                endFrm = int(nums[1])
                depthFilePath = os.path.join(depth_path,depthFileDir)
                depth_data.extend(depth_xml_to_arr(depthFilePath,beginFrm, endFrm, label))
        
    return depth_data

## Load Data  

In [ ]:
"""
Will update soon
"""

## Load Model

In [3]:
model = ln.models.YoloFusion(); model

YoloFusion(
  (layers): ModuleList(
    (0): Sequential(
      (fuse): Conv2d(4, 3, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1_convbatch): Conv2dBatchReLU(3, 32, kernel_size=3, stride=1, padding=1, LeakyReLU(negative_slope=0.1))
    )
    (1): Fusion(
      (Combined): Sequential(
        (2_max): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (3_convbatch): Conv2dBatchReLU(32, 64, kernel_size=3, stride=1, padding=1, LeakyReLU(negative_slope=0.1))
        (4_max): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (5_convbatch): Conv2dBatchReLU(64, 128, kernel_size=3, stride=1, padding=1, LeakyReLU(negative_slope=0.1))
        (6_convbatch): Conv2dBatchReLU(128, 64, kernel_size=1, stride=1, padding=0, LeakyReLU(negative_slope=0.1))
        (7_convbatch): Conv2dBatchReLU(64, 128, kernel_size=3, stride=1, padding=1, LeakyReLU(negative_slope=0.1))
        (8_max): MaxPool2d(kernel_size=2, stride=2, padding=0,